# LLM Over REST

This notebook explains how to use the LLM Over REST integration with LangChain. There is no additional dependencies that you would need to install. It uses `httpx` under the hood, if it is available, otherwise falling back to `requests`.


## Configuration Parameters

Most of basic HTTP configurations are available as parameters during initialization of the LLM object. There is a provision to pass a Callable to create the JSON body for the REST API from the prompt, stop-words and other model parameters. By default, it adds the prompt and stop-words as keys `prompt` and `stop` respectively at the root of the JSON, and appends the entire model parameters object to the JSON. An example of how to provide a custom implementation is provided below.

## Advanced SSL settings

SSL settings are controlled via `ssl_verify` and `client_certs` parameters. 

### Disabling Server Certificate Check

You can set the parameter `ssl_verify` to `False` in order to disable validating the server certificate. In this case, even basic checks like Certificate expiry, Certificate and Server name mismatches are also ignored. **This mode is not recommended for production use as it makes your application vulnerable to Man-in-the-Middle attacks**

### Adding Custom CAs
`ssl_verify` can be set to the path for the CA certificate file in case you want to use custom Certificate Authorities to validate certificates. If the path is a folder it should be rehashed using `openssl rehash`

### Using Client Certificates for mTLS
To authenticate the client with the server via mTLS, you can set the parameter `client_certs` parameter. It can either be a string to the path for the client certificate or a two-Tuple containing the path to the client certificate and the *unencrypted* private key file for the client.

## Generating the JSON body
Users can provide a Callable to generate the JSON body for the request. The method takes three parameters - a prompt string, an optional list of stop words and a dictionary containing all other model parameters. By default, the JSON Body is structured like:

```
{
    "prompt": prompt,
    "stop": stop,
}.update(model_kwargs)
```

The callable should return a `Dict[str, Any]` by combining all the parameters.


In [1]:
from langchain.llms.llm_over_rest import LLMOverREST
from typing import Optional, List, Dict, Any
import json

In [2]:
def create_json(prompt: str, 
                stop: Optional[List[str]] = None, 
                model_kwargs: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    try:
        model_params = model_kwargs.get("parameters") or {}
        model_kwargs["prompt"] = prompt
        if stop and len(stop) > 0:
            model_params["stop"] = stop
    except BaseException as e:
        raise ValueError(f"Error in creating json: {e}")

    return model_kwargs


In [1]:
headers = {
    "X-Auth-Key": secrets.get_password("auth-key"),
    "Content-Type": "application/json"
}

model_params = {
  "model": "highly-secret-llm",
  "parameters": {
    "max_new_tokens": 2023,
    "return_full_text": True,
    "temperature": 0.8,
    "top_p": 0.95,
  }
}

llm = LLMOverREST(api_endpoint="https://llmgateway.mycompany.com/v1/generate",
                  method="POST",
                  headers=headers,
                  ssl_verify="/etc/ssl/certs/ca-certificates",
                  model_kwargs=json_body,
                  json_body_creator=create_json_wmt_endpoint
                 )
response = llm("Tell me something highly secret")
print(response)

The world did not end in 2012 as the Mayans predicted.🤫
